# Creating a netCDF file with eNd for atmospheric flux- distinct eNd boxes
# Part 3- > converting the dust map of all the dust regions from the land mask being zero to being nan
netCDF file on a FAMOUS grid, containing eNd values across the ocean for the atmospheric flux
setting whole ocean to a background eNd value of -8.0 

Following Rempfer et al. 2011 dust boxes

North Atlantic >50 °N eNd = -15
Atlantic < or equal to 50 °N eNd = -12
North Pacific >44 °N eNd = -5
Indopacific < or equal to 44 °N eNd = -7
Remainder eNd= -8

creating a .nc file which i will translate into an ancil file 
from which I will create a mod to convert this into a dust flux of 144Nd and 143Nd

based on examples from:
https://iescoders.com/2017/10/03/writing-netcdf4-data-in-python/
http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/11_create_netcdf_python.pdf

In [1]:
# load packages

%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
import matplotlib.gridspec as gridspec # subplots
from matplotlib import colors as mcolors

import iris
import iris.plot as iplt
import iris.quickplot as qplt
#plt.interactive(True)

import numpy as np
import pandas as pd

import cf_units as cf


## Importing a salinity netCDF and writing to a new netCDF with dust flux boxes

In [2]:
from netCDF4 import Dataset

#this is an nc file with salinity from which to base my template on
f=Dataset('/nfs/see-fs-01_users/ee14s2r/working/surf_files/eNd_dust_mask/dust_boxes_eNd_LandMask.nc','r')

#latitude in and longitude in
latin = f.variables['lat'][:]
latsize=len(latin)
lonin = f.variables['lon'][:]
lonsize=len(lonin)
time=f.variables['time'][:]

#if xconv qrparm.waterfix.famous.nc then it will have the field number
waterflux=f.variables['final'][:]


#Setting the background to nan
#LandMask1=np.where(final<0.5,np.nan)



#open a new netCDF file to create my file for inputing 144Nd surface flux
dataset= Dataset('/nfs/see-fs-01_users/ee14s2r/working/surf_files/eNd_dust_mask/dust_boxes_final.nc', 'w',
                 format='NETCDF4_CLASSIC')

#print(dataset.file_format)
#print(LandMask1)
#print(np.shape(LandMask1))

In [3]:
print(f.dimensions)

OrderedDict([(u'depth', <type 'netCDF4._netCDF4.Dimension'>: name = 'depth', size = 1
), (u'time', <type 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1
), (u'lat', <type 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73
), (u'lon', <type 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 98
)])


In [4]:
print(latin)

[-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90. ]


In [5]:
#Define dimensions used for my variables
depth = dataset.createDimension("depth", 1)
lat = dataset.createDimension("lat", latsize)
lon = dataset.createDimension("lon", lonsize)
time = dataset.createDimension("time", 1)

In [6]:
#Checking dimensions
print (len(lon))

98


## Creating variables with two dimensions (lon and lat) 

In [7]:
#Creating a variable
depth = dataset.createVariable("depth", np.int32, ("depth",))
depth.units = 'm'
depth.positive= 'down'
lat = dataset.createVariable("lat", np.float32, ("lat",))
lat.units = 'degrees_north'
lat.point_spacing = 'even'
lat.long_name = 'latitude'
lon = dataset.createVariable("lon", np.float32, ("lon",))
lon.units = 'degrees_east'
lon.point_spacing = 'even'
lon.long_name = 'longitude'
time = dataset.createVariable("time", np.float64, ("time",))
time.units = ('days since 0000-01-01 00:00.00') 

In [8]:
#writing data and slicing it to lon and lat
lat[:]= latin
lon[:]= lonin

print(lat[1])

-87.5


## Creating variable for Background eNd (-8)  

In [9]:
#This np.where is changing values of salnity into background eNd values and then changing the land mask to 0
#waterflux3=np.where(waterflux>1.0E20,-8.0,-8.0)

#Create the 2d-variable for backgrond and land mask
background = dataset.createVariable("background", np.float32, ("time","depth","lat","lon"))
#background.units = "ratio"
background.long_name = "background_Epsilon_Nd"

#filling the background eNd values
background[:,:,:,:]=waterflux

In [10]:

#background[background==0.0]= np.nan

#background2=np.where(background==0.0,'nan', background)

In [11]:
#backgroundNaN = background

#backgroundNaN[backgroundNaN== 0.0] = np.nan

background2= np.where(background[:,:,:,:]==0.0,np.nan,background)

In [12]:
#This np.where is changing values of salnity into background eNd values and then changing the land mask to 0
#waterflux3=np.where(waterflux>1.0E20,-8.0,-8.0)

#Create the 2d-variable forland mask
eNd_nan = dataset.createVariable("eNd_nan", np.float32, ("time","depth","lat","lon"))
#background.units = "ratio"
eNd_nan.long_name = "Dust_Regions_eNd"

#filling the background eNd values
eNd_nan[:,:,:,:]=background2

In [13]:
dataset.close()